In [1]:
import numpy as np
import cv2 as cv

In [2]:
cap = cv.VideoCapture(r'C:\Users\BlackySwanny\Downloads\video_2025-12-16_03-45-11.mp4')

In [3]:
ret, first_frame = cap.read()

In [4]:
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)

In [5]:
mask = np.zeros_like(first_frame)

In [6]:
mask[..., 1] = 255

In [7]:
while(cap.isOpened()):
    
    # ret = a boolean return value from getting
    # the frame, frame = the current frame being
    # projected in the video
    ret, frame = cap.read()
    
    # Opens a new window and displays the input
    # frame
    cv.imshow("input", frame)
    
    # Converts each frame to grayscale - we previously 
    # only converted the first frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Farneback parameters (balanced defaults)
    pyr_scale = 0.5
    levels = 3
    winsize = 15
    iterations = 3
    poly_n = 5
    poly_sigma = 1.2
    flags = 0
    
    # Calculates dense optical flow by Farneback method
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags)
    
    # Computes the magnitude and angle of the 2D vectors
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    
    # Sets image hue according to the optical flow 
    # direction
    mask[..., 0] = angle * 180 / np.pi / 2
    
    # Sets image value according to the optical flow
    # magnitude (normalized)
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    
    # Converts HSV to RGB (BGR) color representation
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
    
    # Opens a new window and displays the output frame
    cv.imshow("dense optical flow", rgb)
    
    # Updates previous frame
    prev_gray = gray
    
    # Frames are read by intervals of 1 millisecond. The
    # programs breaks out of the while loop when the
    # user presses the 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# The following frees up resources and
# closes all windows
cap.release()
cv.destroyAllWindows()